In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import requests 
from bs4 import BeautifulSoup 
import json

In [2]:
# pull data from google sheets
def sheetsync(datasheet,tab):
    
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
             'qd-cnov-database-2027ad01e8de.json', scope) 
    gc = gspread.authorize(credentials)
    
    spreadsheetName = datasheet
    sheetName = tab  # <--- please set the sheet name here.
    spreadsheet = gc.open(spreadsheetName)
    sheet = spreadsheet.worksheet(sheetName)
    
    data = sheet.get_all_values()
    headers = data.pop(0)

    df = pd.DataFrame(data, columns=headers)
    return df
sheetsync('dataset','qd')

,确诊案例,年龄,性别,确诊日期,去过武汉,武汉人/长居武汉,治愈,定点医院,医院市区,居民市区,...,到达时间,出发地,到达地,出发市,到达市,transportation,transportation no.,seat no,detailed description,link
0,1,37,男,01-21-2020,是,是,是,青岛大学附属医院,,市南区,...,,日照,青岛,日照,青岛,雇车\n,,,37岁男性，武汉人，在日照市工作。因发热等症状，于1月17日到日照市就诊，当晚自行至青岛市就诊。,http://wsjsw.qingdao.gov.cn/n28356065/n3256306...
1,2,41,男,01-23-2020,是,否,是,青岛大学附属医院,,李沧区\n,...,\n,青岛李沧区,武汉,李沧\n,武汉,,,,我市李沧区41岁男性患者，1月13日赴武汉，1月14日返青，1月16日出现发热症状，1月21...,http://wsjsw.qingdao.gov.cn/n28356065/n3256306...
2,2,,,,,,,,,,...,,武汉,青岛,武汉,青岛,,,,,
3,3,66,男,01-23-2020,是,是,,,,市北区\n,...,,鄂州,青岛,鄂州,青岛,,,,湖北鄂州66岁男性患者，1月14日来青，暂住市北区，1月14日出现发热症状，1月19日到我市...,http://wsjsw.qingdao.gov.cn/n28356065/n3256306...
4,4,24,男,01-24-2020,是,否,是,青岛大学附属医院,,黄岛区,...,,即墨区,武汉,即墨,武汉,,,,我市24岁男性患者，黄岛人。1月10日到武汉出差，11日返回工作地即墨，21日开始发热伴呼吸...,http://wsjsw.qingdao.gov.cn/n28356065/n3256306...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,55,33,男,02-14-2020,否,否,,,李沧区,市北区\n,...,,泰安,青岛,,,火车,K68,,33岁男性，常住市北区登州路街道丹山路66号。1月30日乘K68次列车自泰安至青岛，2月11...,http://wsjsw.qingdao.gov.cn/n28356065/n3256920...
63,56,90,女,02-14-2020,否,否,,,,市南区,...,,,,,,,,,患者王某某，90岁女性，常住市南区四川路68号。系病例54的密切接触者，经市疾控中心核酸检测...,http://wsjsw.qingdao.gov.cn/n28356065/n3256306...
64,57,78,女,02-15-2020,否,否,,,,市北区,...,,,,,,,,,患者王某某，78岁女性，常住市北区即墨路街道海逸景园。系我市确诊病例54的密切接触者，2月1...,http://wsjsw.qingdao.gov.cn/n28356065/n3256306...
65,58,56,女,02-17-2020,否,否,,,,市北区,...,,,,,,,,,患者徐某某，56岁女性，常住市南区四川路68号，系我市确诊病例54的密切接触者、病例56的女...,http://wsjsw.qingdao.gov.cn/n28356065/n3256306...


### Crawl bulletin from the website

In [130]:

def start_requests(url):
    print(url) 
    r = requests.get(url)
    return r.content

# 第一级 links 
def get_page(text):
    soup = BeautifulSoup(text, 'html.parser')
    links = soup.find_all('a', class_ = 'nrf')
    pages = []
    for link in links:
        url = 'http://wsjsw.qingdao.gov.cn/'+link.get('href')
        pages.append(url)
    return pages

# 第二级 links and get info
def parse_page(text):
    soup = BeautifulSoup(text, 'html.parser')
    mydict = {}
    time = soup.find('div', class_ = 'fbbt-wenz').text.strip()
    if time > '2020-01-21':
        
        mydict['time'] = soup.find('div', class_ = 'fbbt-wenz').text
        mydict['parag'] = soup.find('div', class_ = 'neirong-wz').text
    else:return mydict
    return mydict


def write_json(result):
    s = json.dumps(result, indent = 4, ensure_ascii=False)
    with open('data/bulletin.json', 'w', encoding = 'utf-8') as f:
        f.write(s)
def crawl(url):
    text = start_requests(url)
    pageurls = get_page(text) 
    for pageurl in pageurls: 
        page = start_requests(pageurl)
        mydict = parse_page(page)
        result_list.append(mydict)
def main():
    url = 'http://wsjsw.qingdao.gov.cn/n28356065/n32563060/n32563061/index.html'
    crawl(url)
    for i in range(2, 5):
        url = 'http://wsjsw.qingdao.gov.cn/n28356065/n32563060/n32563061/index_{}.html'.format(i)
        crawl(url)
#         text = start_requests(url)
#         pageurls = get_page(text) 
#         for pageurl in pageurls: # 
#             page = start_requests(pageurl)
#             mydict = parse_page(page)
#             result_list.append(mydict)
    write_json(result_list) 

if __name__ == '__main__':
    result_list = []
    main()

http://wsjsw.qingdao.gov.cn/n28356065/n32563060/n32563061/index.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200207091904921858.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200206141301155174.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200206090847843337.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200205152144155256.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200205084010452857.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200204155030655835.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200204092147343110.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200203144006609133.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200203092831140802.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200202152750421844.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200202102259593635.html
http://wsjsw.qingd

In [131]:
with open('data/bulletin.json',encoding="utf8") as f:
    data = json.load(f)
data

[{'time': ' \xa0 \xa0\xa0\xa0  2020-02-07',
  'parag': '\n2020年2月6日12—24时，我市新增2例新型冠状病毒感染的肺炎确诊病例。病例42：51岁男性，现住崂山区，医务人员。1月19日乘A67107航班由昆明返回青岛（同机乘客中目前已报告多名确诊病例，其中包括我市病例7）。1月25日出现发热症状，自服药1日后恢复。2月5日到市北区某医院发热门诊就诊，经该院核酸检测和专家组评估，2月6日下午确诊。病例43：51岁女性，现住崂山区，医务人员，与病例42系夫妻关系，1月19日与病例42同乘A67107航班由昆明返回青岛。2月5日出现发热症状，当天与病例42一同到市北区某医院发热门诊就诊，经该院核酸检测和专家组评估，2月6日下午确诊。现已对以上2例确诊病例的17名密切接触者开展医学观察，目前无发热等异常情况。其他密切接触者正在进一步追踪和调查中。截至2月6日24时，我市累计确诊43例新型冠状病毒感染的肺炎病例，其中重症4例，危重症1例，治愈出院4例，患者均在定点医院接受隔离治疗。累计已排除疑似病例101例，现有疑似病例35例，正在隔离检测治疗。累计已解除医学观察254人，仍纳入医学观察的密切接触者244人，未发现异常。目前按病例确诊医院所在区市分布：市南区5例（已治愈出院1例）、市北区8例、李沧区4例、崂山区2例（已治愈出院1例）、城阳区2例（已治愈出院1例）、黄岛区9例（已治愈出院1例）、即墨区8例、胶州市1例、平度市3例、莱西市1例。\xa0\xa0青岛市卫生健康委员会2020年2月7日\xa0\n'},
 {'time': ' \xa0 \xa0\xa0\xa0  2020-02-06',
  'parag': '\n2020年2月6日0—12时，我市无新增新型冠状病毒感染的肺炎确诊病例，新增治愈出院1例。截至2月6日12时，我市累计确诊41例新型冠状病毒感染的肺炎病例，其中重型3例，危重型2例，治愈出院4例，患者均在定点医院接受隔离治疗。累计已排除疑似病例99例，现有疑似病例23例，正在隔离检测治疗。累计已解除医学观察244人，仍纳入医学观察的密切接触者267人，未发现异常。目前按病例确诊医院所在区市分布：市南区5例（已治愈出院1例）、市北区6例、李沧区4例、崂山区2例（已治愈出院1例）、城阳区2例（已治

### Get Diagonosed data

In [132]:
df = pd.read_json('data/bulletin.json',encoding="utf8")
df.head()

,time,parag
0,2020-02-07,\n2020年2月6日12—24时，我市新增2例新型冠状病毒感染的肺炎确诊病例。病例42：5...
1,2020-02-06,\n2020年2月6日0—12时，我市无新增新型冠状病毒感染的肺炎确诊病例，新增治愈出院1例...
2,2020-02-06,\n2020年2月5日12—24时，我市新增6例新型冠状病毒感染的肺炎确诊病例。病例36：5...
3,2020-02-05,\n 2020年2月5日0—12时，我市新增1例新型冠状病毒感染的肺炎确诊病例。 新增确...
4,2020-02-05,\n 2020年2月4日12—24时，我市新增2例新型冠状病毒感染的肺炎确诊病例。 新增...


In [148]:
def save_data(stype):
    """
    stype: 'cured','diag'

    """
    test = df.parag[0].split('分布：')[1].split('。')[0]
    if test:
        test = test.split('、')
    cured = {}
    for i,t in enumerate(test):

        if ('已治愈出院') in t:
            cured[str(t[:3])]= t.split('已治愈出院')[1][0]
            test[i] = t.split('（已治愈出院')[0]
    test = ''.join(test)
    # list of diagnosed data 
    import re
    temp = re.findall(r'\d+', test) 
    res = list(map(int, temp))

    if stype == 'cured':
        save_cured(cured)
    elif stype == 'diag':
        save_diag(res)
        
def save_cured(cured):
    with open('data/cured.json', 'w', encoding = 'utf-8') as f:
        json.dump(cured, f)
def save_diag(res):
    district = ['市南区','市北区','李沧区','崂山区','城阳区','黄岛区','即墨市','胶州市','平度市','莱西市']
    diag = [list(z) for z in zip(district,res)]
    with open('data/diag.json', 'w', encoding = 'utf-8') as f:
        json.dump(diag, f)


{'市南区': '1', '崂山区': '1', '城阳区': '1', '黄岛区': '1'}


In [149]:
save_data('cured')
save_data('diag')

## get clinic location

In [ ]:
sheetsync('dataset','clinic')

In [8]:
f = open("data/access.txt", "r")
access = f.read()

In [117]:
data = sheetsync('dataset','clinic')
data.head()

,行政区划,机构名称,机构地址,lng,lat
0,市南区,青岛大学附属医院（市南院区）,青岛市市南区江苏路16号,120.333868,36.07301807
1,市南区,青岛市市立医院（东院区）,青岛市市南区东海中路5号,120.4087659,36.06437495
2,市南区,山东青岛中西医结合医院,青岛市市南区嘉祥路3号,120.3124275,36.07142395
3,市南区,青岛市第九人民医院,青岛市市南区朝城路2号甲,120.3170087,36.06651808
4,市南区,青岛市市南区人民医院,青岛市市南区广州路29号,120.3177762,36.07217355


In [104]:
import json
def get_lnglat(location):
    
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'} 
    for i,loc in enumerate(data['机构地址']):
        pa = {
            'address': loc, 
            #'callback':'showLocation',
            'output': 'json',
            'ak': access
        }
        r = requests.get("http://api.map.baidu.com/geocoding/v3/", params=pa, headers= headers)
        rejs = json.loads(r.text)
        data['lng'][i]=rejs['result']['location']['lng']
        data['lat'][i]=rejs['result']['location']['lat'] 

    return data

In [116]:
data

,行政区划,机构名称,机构地址,lng,lat
0,市南区,青岛大学附属医院（市南院区）,青岛市市南区江苏路16号,120.334,36.073
1,市南区,青岛市市立医院（东院区）,青岛市市南区东海中路5号,120.409,36.0644
2,市南区,山东青岛中西医结合医院,青岛市市南区嘉祥路3号,120.312,36.0714
3,市南区,青岛市第九人民医院,青岛市市南区朝城路2号甲,120.317,36.0665
4,市南区,青岛市市南区人民医院,青岛市市南区广州路29号,120.318,36.0722
5,市北区,青岛山大齐鲁医院,青岛市市北区合肥路758号,120.428,36.1188
6,市北区,青岛市市立医院,青岛市市北区胶州路1号,120.334,36.0797
7,市北区,青岛市中心医院,青岛市市北区四流南路127号,120.383,36.1456
8,市北区,青岛市中医医院,青岛市市北区人民路4号,120.367,36.1008
9,市北区,青岛市妇女儿童医院,青岛市市北区辽阳西路217号,120.409,36.1028


## 定点医院名称，地址，经纬度

In [121]:
data = sheetsync('dataset','clinic')

lng = data['lng'].astype(float).values.tolist()

lat = data['lat'].astype(float).values.tolist()
coord = [list(z) for z in zip(lng,lat)]
result = {}
for i,r in data.iterrows():
    result[str(data['机构名称'][i])]=  coord[i]
    
    
# save to local
with open('data/clinic-coord.json', 'w') as fp:
    json.dump(result, fp)

dict

## Save the datasheets to local 

In [158]:
data = sheetsync('dataset','qd')
data.to_csv('data/dataset.csv')
data.head()

,确诊案例,年龄,性别,确诊日期,去过武汉,武汉人/长居武汉,治愈,定点医院,市区,行程日期,出发时间,到达时间,出发地,到达地,transportation,transportation no.,seat no,detailed description,link
0,1,37,男\n,01-21-2020,是,是,是,青岛大学附属医院,市南区,01-17-2020,,,日照,青岛\n,雇车\n,,,37岁男性，武汉人，在日照市工作。因发热等症状，于1月17日到日照市就诊，当晚自行至青岛市就诊。,http://wsjsw.qingdao.gov.cn/n28356065/n3256306...
1,2,41,男\n,01-23-2020,是,否,,,李沧区\n,01-13-2020,,\n,青岛李沧区,武汉,,,,我市李沧区41岁男性患者，1月13日赴武汉，1月14日返青，1月16日出现发热症状，1月21...,http://wsjsw.qingdao.gov.cn/n28356065/n3256306...
2,2,,,,,,,,,01-14-2020,,,武汉,青岛,,,,,
3,3,66,男,01-23-2020,是,是,,,市北区\n,01-14-2020,,,鄂州\n,青岛,,,,湖北鄂州66岁男性患者，1月14日来青，暂住市北区，1月14日出现发热症状，1月19日到我市...,http://wsjsw.qingdao.gov.cn/n28356065/n3256306...
4,4,24,男,01-24-2020,是,否,,,黄岛区,01-10-2020,,,即墨区,武汉\n,,,,我市24岁男性患者，黄岛人。1月10日到武汉出差，11日返回工作地即墨，21日开始发热伴呼吸...,http://wsjsw.qingdao.gov.cn/n28356065/n3256306...


,确诊案例\n,年龄,性别\n,确诊日期,去过武汉\n,武汉人/长居武汉,治愈\n,定点医院\n,市区\n,行程日期\n,出发时间\n,到达时间\n,出发地\n,到达地\n,transportation,transportation no.,seat no,detailed description,link
0,1,37,男\n,01-21-2020,是,是,是,青岛大学附属医院,市南区,01-17-2020,,,日照,青岛大学附属医院,雇车\n,,,37岁男性，武汉人，在日照市工作。因发热等症状，于1月17日到日照市就诊，当晚自行至青岛市就诊。,http://wsjsw.qingdao.gov.cn/n28356065/n3256306...
1,2,41,男\n,01-23-2020,是,否,,,李沧区\n,01-13-2020,,\n,青岛李沧区,武汉,,,,我市李沧区41岁男性患者，1月13日赴武汉，1月14日返青，1月16日出现发热症状，1月21...,http://wsjsw.qingdao.gov.cn/n28356065/n3256306...
2,2,,,,,,,,,01-14-2020,,,武汉,青岛,,,,,
3,3,66,男,01-23-2020,是,是,,,市北区\n,01-14-2020,,,鄂州\n,青岛,,,,湖北鄂州66岁男性患者，1月14日来青，暂住市北区，1月14日出现发热症状，1月19日到我市...,http://wsjsw.qingdao.gov.cn/n28356065/n3256306...
4,4,24,男,01-24-2020,是,否,,,黄岛区,01-10-2020,,,即墨,武汉\n,,,,我市24岁男性患者，黄岛人。1月10日到武汉出差，11日返回工作地即墨，21日开始发热伴呼吸...,http://wsjsw.qingdao.gov.cn/n28356065/n3256306...
